In [1]:
import keras
from keras.layers import *
from keras.models import Model,Sequential
import os
import numpy as np
import cv2

Using TensorFlow backend.


In [2]:
image_path='/Users/hit.flouxetine/datasets/dataset1/images_prepped_train'
anno_path='/Users/hit.flouxetine/datasets/dataset1/annotations_prepped_train'
image_height, image_width=360, 480
lr=0.001
n_classes=11

In [3]:
x_train=[]
y_train=[]
for elem in sorted(os.listdir(image_path)):
    file_name=image_path+'/'+elem
    image=cv2.imread(file_name)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    x_train.append(image)
    
    anno_name=anno_path+'/'+elem
    anno=cv2.imread(anno_name)
    anno=cv2.cvtColor(anno,cv2.COLOR_BGR2RGB)
    anno=anno.reshape((image_width*image_height,3))
    anno=anno[...,0]
    y_truth=np.zeros((image_width*image_height,n_classes))
    for i in range(anno.shape[0]):
        y_truth[i][anno[i]-1]=1
    y_train.append(y_truth)

y_train=np.array(y_train,dtype='float32')
x_train=np.array(x_train,dtype='float32')
(x_train.shape,y_train.shape)

((367, 360, 480, 3), (367, 172800, 11))

In [4]:
input_layer=Input(shape=(image_height,image_width,3))
model=Conv2D(kernel_size=(3,3),filters=64, padding='same', activation='relu')(input_layer)
model=Conv2D(kernel_size=(3,3),filters=64, padding='same', activation='relu')(model)
model=MaxPool2D(pool_size=(2,2), strides=2)(model)
block1=model
model=Conv2D(kernel_size=(3,3),filters=128,padding='same', activation='relu')(model)
model=Conv2D(kernel_size=(3,3),filters=128,padding='same', activation='relu')(model)
model=MaxPool2D(pool_size=(2,2),strides=2)(model)
block2=model
model=Conv2D(kernel_size=(3,3),filters=256,padding='same', activation='relu')(model)
model=Conv2D(kernel_size=(3,3),filters=256,padding='same', activation='relu')(model)
model=MaxPool2D(pool_size=(2,2),strides=2)(model)
block3=model
model=Conv2D(kernel_size=(3,3),filters=512,padding='same', activation='relu')(model)
model=Conv2D(kernel_size=(3,3),filters=512,padding='same', activation='relu')(model)
model=Conv2D(kernel_size=(3,3),filters=512,padding='same', activation='relu')(model)
model=MaxPool2D(pool_size=(2,2),strides=2)(model)
block4=model
model=Conv2D(kernel_size=(3,3),filters=512,padding='same', activation='relu')(model)
model=Conv2D(kernel_size=(3,3),filters=512,padding='same', activation='relu')(model)
model=Conv2D(kernel_size=(3,3),filters=512,padding='same', activation='relu')(model)
model=ZeroPadding2D(padding=((1,1),(0,0)))(model)
model=MaxPool2D(pool_size=(2,2),strides=2)(model)

model=Conv2D(kernel_size=(3,3),filters=4096,padding='same', activation='relu')(model)
model=Conv2D(kernel_size=(1,1),filters=4096,padding='same', activation='relu')(model)
model=Conv2D(kernel_size=(1,1),filters=n_classes,activation='relu')(model)
block5=model

block5= Conv2DTranspose(kernel_size=(2,2),strides=2,filters=n_classes)(block5)
block4= Conv2D(kernel_size=(1,1),filters=n_classes)(block4)
block4= ZeroPadding2D(padding=((1,1),(0,0)))(block4)
model=Add()([block5,block4])
block5=model
block5= Conv2DTranspose(kernel_size=(2,2),strides=2,filters=n_classes)(block5)
block5= Cropping2D(cropping=((1,0),(0,0)))(block5)
block3= Conv2D(filters=n_classes,kernel_size=(1,1))(block3)
block3= ZeroPadding2D(padding=((1,1),(0,0)))(block3)
model=Add()([block5,block3])

model=Conv2DTranspose(filters=n_classes,kernel_size=(8,8),strides=8)(model)
model=Cropping2D(cropping=((8,8),(0,0)))(model)
model=Reshape((image_height*image_width,n_classes))(model)
model=Activation('softmax')(model)
model=Model(inputs=input_layer,outputs=model)


In [5]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
model.fit(x_train,y_train,epochs=5,batch_size=64)

Epoch 1/5
367/367 [==============================] - 966s 3s/step - loss: 3.3923 - accuracy: 0.0932
Epoch 2/5
367/367 [==============================] - 937s 3s/step - loss: 2.3571 - accuracy: 0.1341
Epoch 3/5
367/367 [==============================] - 1027s 3s/step - loss: 2.2856 - accuracy: 0.1723
Epoch 4/5
367/367 [==============================] - 1069s 3s/step - loss: 2.1582 - accuracy: 0.2195
Epoch 5/5
367/367 [==============================] - 1010s 3s/step - loss: 1.9874 - accuracy: 0.2592
